In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from causallearn.search.ScoreBased.GES import ges
from causallearn.search.ConstraintBased.PC import pc
import itertools
from sklearn.preprocessing import StandardScaler
from dagma.nonlinear import DagmaMLP, DagmaNonlinear
from dagma.linear import DagmaLinear
from castle.algorithms import NotearsNonlinear, DAG_GNN
from notears.notears.nonlinear import NotearsMLP, notears_nonlinear
import pytetrad.tools.TetradSearch as ts
import torch
import os
import pickle
import networkx as nx
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

/homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-24 13:27:40,504 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2025-04-24 13:27:40,628 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


ImportError: Unable to import 'edu.cmu.tetrad.data.DataSet' due to incorrect Java version

In [2]:
dose = "A"
df = pd.read_csv(f"data/huvec/cd_matrix_d{dose}.csv")
df_gene_expression = df.drop(columns=["radiation"])
num_genes = len(df_gene_expression.columns)
gene_names = df_gene_expression.columns
df_gene_expression = StandardScaler().fit_transform(df_gene_expression)
df_gene_expression = pd.DataFrame(data=df_gene_expression)
df_learn = pd.DataFrame(data=df_gene_expression)
df_learn.columns = gene_names
df_learn['radiation'] = df['radiation']
 

In [3]:
print(df)

    MYORG  WDR44  NSA2  MRPS26  PTGS2  IFT52  MAF  LOC105377627  DAPK3  MKNK2  \
0    1140   1618  3404    1157   2288   1550  128           190   5910   3456   
1    1402   2213  5139    1574   1413   2219  118            73   5513   8203   
2    1123   1785  4566    1190   1381   2008  138           252   5579   3954   
3    1126   1386  3888    1012    539   1954   70           172   5282   3925   
4     836   1487  2848     828   2276   1246   68           188   4821   2950   
5    1390   2009  4752    1514   1800   2316  218           379   6659   4280   
6    1262   1530  3964    1040    696   1934   88           201   5233   4082   
7     994   1532  4000    1282   1587   1819   28           134   4505   4895   
8    1110   1863  4550    1371   1411   2260   28            84   5282   5687   
9     966   1843  5222    1424   1148   2556   48           119   5360   6291   
10    865   1613  4360    1270    894   2186   54            96   4510   5327   
11   1024   1594  5256    13

In [8]:
#device = torch.device("mps" if torch.mps.is_available() else "cpu")
def dagma_nonlinear_local_learn(subproblem, use_skel):
    skel, data = subproblem
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    d = data.shape[1]
    X = data.values
    X_tensor = torch.from_numpy(X).type(torch.double).to(device)
    eq_model = DagmaMLP(dims=[d, np.min([10, d-1]), 1], bias=True, dtype=torch.double) # create the model for the structural equations, in this case MLPs
    eq_model = eq_model.to(device)
    model = DagmaNonlinear(eq_model, dtype=torch.double) # create the model for DAG learning
    # adj = model.fit(X_tensor, lambda1=0.02, lambda2=0.005) # fit the model with L1 reg. (coeff. 0.02) and L2 reg. (coeff. 0.005)
    adj = model.fit(X_tensor, lambda1=0.001, lambda2=0.0001) # fit the model with L1 reg. (coeff. 0.02) and L2 reg. (coeff. 0.005)

    return adj

def ges_non_param_local_learn(subproblem, use_skel):
    skel, data = subproblem
    data = data.drop(columns=['target'])
    print(data.columns)
    result = ges(data.values, maxP=10, node_names=data.columns, score_func="local_score_CV_general")
    return result
def pc_kci_local_learn(subproblem, use_skel):
    skel, data = subproblem
    data = data.drop(columns=['target'])
    result = pc(data.values, alpha=0.05, indep_test='kci')
    return result
def dag_gnn_local_learn(subproblem, use_skel):
    skel, data = subproblem
    d = data.shape[1]
    X = data.values
    model = DAG_GNN(device_type="gpu")
    model.learn(X)
    return model.causal_matrix
    
def notears_mlp_local_learn(subproblem, use_skel):
    skel, data = subproblem
    d = data.shape[1]
    X = data.values
    model = NotearsNonlinear(device_type="gpu")
    adj = model.learn(X)
    # model = NotearsMLP(dims=[d, 10, 1], bias=True)
    # #print(model.fc1_neg.weight.device, X.device)
    # adj = notears_nonlinear(model, X, lambda1=0.01, lambda2=0.01, device='gpu')
    return adj

### Global learning 

In [5]:
ground_truth = nx.read_gexf(f"./data/huvec/cd_subgraph_d{dose}.gexf" )
print(ground_truth)

DiGraph with 918 nodes and 1670 edges


In [6]:
from GENIE3 import GENIE3
# global_adj = ges_local_learn([superstructure, df], use_skel=True)
#global_adj = dagma_nonlinear_local_learn([None, df_learn], use_skel=False)
#global_adj = dag_gnn_local_learn([None, df_learn.iloc[:,0:1000]], use_skel=False)
#global_adj = notears_mlp_local_learn([None, df_learn.iloc[:,0:1000]], use_skel=False)
#print(np.sum(global_adj!=0))
global_adj = GENIE3(df_learn[list(ground_truth.nodes)+['radiation']].values, nthreads=16)


Tree method: RF
K: sqrt
Number of trees: 1000


running jobs on 16 threads
Elapsed time: 59.69 seconds


In [8]:
import networkx as nx
genie_net = nx.from_numpy_array(global_adj, create_using=nx.DiGraph)
genie_net = nx.relabel_nodes(genie_net, dict(zip(np.arange(len(genie_net.nodes)), list(ground_truth.nodes)+['radiation'])))
print(genie_net)

DiGraph with 919 nodes and 842087 edges


In [9]:
true_positive = 0
for edge in ground_truth.edges:
    if genie_net.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)
print(len(list(genie_net.neighbors("radiation"))))
sorted_nodes = sorted(genie_net.out_degree(), key=lambda x: x[1], reverse=True)

# Print nodes and their degree
print("Nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")

('B2M', 'CALR')
('B2M', 'VAV2')
('B2M', 'LNPEP')
('B2M', 'CD9')
('B2M', 'TAPBPL')
('B2M', 'TFRC')
('B2M', 'DKK3')
('B2M', 'GYPC')
('B2M', 'ARG2')
('B2M', 'KPNA2')
('B2M', 'MICA')
('B2M', 'HMBS')
('B2M', 'ACE')
('B2M', 'UBC')
('B2M', 'SLC4A11')
('B2M', 'HNRNPC')
('B2M', 'PTPRJ')
('B2M', 'NOTCH1')
('B2M', 'CTSD')
('B2M', 'RANGAP1')
('B2M', 'ATP8A2')
('B2M', 'PIK3R1')
('B2M', 'ATP8A1')
('B2M', 'SEC23A')
('B2M', 'CANX')
('B2M', 'UBB')
('B2M', 'PDIA3')
('B2M', 'DLGAP5')
('B2M', 'PIK3CB')
('B2M', 'ICAM1')
('B2M', 'TPST1')
('B2M', 'FYN')
('B2M', 'FCGRT')
('B2M', 'SLC4A2')
('B2M', 'HFE')
('B2M', 'CBL')
('B2M', 'MAP1S')
('B2M', 'VAV3')
('B2M', 'SLC4A8')
('B2M', 'HLA-A')
('B2M', 'CXCR4')
('B2M', 'CDR1')
('B2M', 'PROCR')
('B2M', 'LIN54')
('B2M', 'MAF')
('B2M', 'SHC1')
('B2M', 'VCP')
('B2M', 'ADRA1D')
('B2M', 'NUSAP1')
('B2M', 'PLCG2')
('B2M', 'PCNA')
('B2M', 'HDAC1')
('B2M', 'CD82')
('B2M', 'SEC24D')
('CALR', 'B2M')
('CCND3', 'CDKN1A')
('CDKN1A', 'HNRNPK')
('CDKN1A', 'SET')
('CDKN1A', 'UHRF2')
('

### Superstructure

In [19]:

# Correlation matrix with Permutation testing
corr_mat = df_gene_expression.corr('pearson', numeric_only=True).to_numpy()
print(np.min(np.abs(corr_mat)))
np.fill_diagonal(corr_mat, 0)
print(np.max(corr_mat), np.argmax(corr_mat))
random_corr_coef = []
for i in range(10):
    shuffled_array = df_gene_expression.values
    [np.random.shuffle(x) for x in shuffled_array]
    shuffled_final_data_set = pd.DataFrame(data=shuffled_array)
    shuffle_corr_mat = shuffled_final_data_set.corr('pearson', numeric_only=True).to_numpy()
    np.fill_diagonal(shuffle_corr_mat, 0)
    print(np.max(shuffle_corr_mat), np.argmax(shuffle_corr_mat))
    random_corr_coef.append(np.max(shuffle_corr_mat))
print(random_corr_coef)
ci_interval = st.t.interval(0.95, len(random_corr_coef)-1, 
                            loc=np.mean(random_corr_coef), 
                            scale=st.sem(random_corr_coef))
print(ci_interval)
cutoff = ci_interval[1]

corr_mat[corr_mat<=cutoff] = 0
corr_mat[corr_mat>cutoff] = 1

superstructure = corr_mat
print(f"Superstructure contains {np.sum(superstructure)} edges which is \
        {np.sum(superstructure)/(superstructure.shape[1]**2)} fraction of all possible edges")

nan
nan 5886
0.9931338554691231 2400661
0.9922982607314539 19666634
0.9885011722553623 18559022
0.9933796287091708 5287612
0.9899321810732494 3587886
0.9894394392309578 9951651


KeyboardInterrupt: 

In [9]:
# STRING PPI
ppi_df = pd.read_csv("./data/prior_knowledge/string_ppi_gene_names.csv", header=0)
string_undirected_edges = {(start, end) for start, end in zip(ppi_df["protein1"], ppi_df["protein2"])}
ppi_superstructure = np.zeros((num_genes, num_genes))
for i, j in itertools.product(np.arange(num_genes), np.arange(num_genes)):
    start = df.columns[i]
    end = df.columns[j]
    if (start, end) in string_undirected_edges:
        ppi_superstructure[i][j] = 1
        
print(f"Superstructure contains {np.sum(ppi_superstructure)} edges which is \
        {np.sum(ppi_superstructure)/(ppi_superstructure.shape[1]**2)} fraction of all possible edges")



Superstructure contains 298183.0 edges which is         0.008606812943491176 fraction of all possible edges


In [8]:
superstructure = ppi_superstructure

### Partition network

In [5]:
with open(f"./data/huvec/cd_partition_d{dose}_new.pickle", 'rb') as f:
    custom_partition = pickle.load(f)
print(custom_partition)
for i, comm in custom_partition.items():
    comm += ['radiation']

{0: ['TAPBPL', 'PDIA3', 'DKK3', 'ETS1', 'FOXP1', 'FLI1', 'SHC1', 'CBL', 'VAV2', 'CTSD', 'ACE', 'CXCR4', 'JUND', 'CD9', 'VAV3', 'E2F1', 'B2M', 'UBC', 'NFIC', 'ATF2', 'CDR1', 'ETS2', 'RANGAP1', 'ELF1', 'LIN54', 'BCOR', 'PLCG2', 'MICA', 'GADD45A', 'VCP', 'FYN', 'PIK3CB', 'HMBS', 'SEC24D', 'CD82', 'WDR5', 'SLC4A2', 'BRD4', 'STAT2', 'MED12', 'KLF4', 'LNPEP', 'HFE', 'LMNB1', 'ATP8A2', 'ADRA1D', 'ATP8A1', 'CANX', 'DLGAP5', 'KPNA2', 'UBB', 'IRF1', 'ERG', 'MAP1S', 'HDAC1', 'BHLHE40', 'SLC4A11', 'STAT5A', 'CALR', 'RAD21', 'ARG2', 'FCGRT', 'GYPC', 'MAF', 'MYC', 'NOTCH1', 'SLC4A8', 'MAZ', 'CDKN1A', 'IRF2', 'PTPRJ', 'PIK3R1', 'TPST1', 'PCNA', 'TFRC', 'HLA-A', 'NUSAP1', 'HNRNPC', 'SEC23A', 'AR', 'ICAM1', 'EGR1', 'ZNF263', 'PROCR'], 1: ['BAG2', 'ETS1', 'REPIN1', 'AGPAT2', 'ATAD3B', 'AXIN2', 'RCC1', 'UTP14A', 'RHOT1', 'PALD1', 'IGF1R', 'JUN', 'UBC', 'STK40', 'UBE2C', 'GPR4', 'TSPAN13', 'WRN', 'SREBF2', 'CLIP1', 'BCOR', 'PA2G4', 'RFC3', 'KIF23', 'N4BP3', 'SRSF3', 'FLCN', 'SSR2', 'RFC4', 'PLSCR1', 'EIF3

In [6]:
def partition_problem(partition: dict, structure: np.ndarray, data: pd.DataFrame):
    sub_problems = []
    k = list(partition.keys())
    k.sort()
    for i in k:
        sub_nodes = partition[i]
        sub_structure = np.ones((len(sub_nodes), len(sub_nodes)))
        sub_nodes = list(sub_nodes)
        sub_data = data[sub_nodes] 
        sub_problems.append((sub_structure, sub_data))
    return sub_problems

### Learning subgraphs

In [9]:
import functools
from concurrent.futures import ProcessPoolExecutor
from cd_v_partition.causal_discovery import ges_local_learn

superstructure = np.ones((num_genes+1, num_genes+1))
subproblems = partition_problem(custom_partition, superstructure, df)
# Setup thread level parallelism
# func_partial = functools.partial(
#     dagma_nonlinear_local_learn,
#     use_skel=False,
# )
# n_comms = len(ec_partition_small)
# nthreads = n_comms  # each thread handles one partition
results = []
# chunksize = max(1, n_comms // nthreads)
for subproblem in subproblems:
    #adj = ges_non_param_local_learn(subproblem, use_skel=False)
    #adj = pc_kci_local_learn(subproblem, use_skel=False)
   # adj = dagma_nonlinear_local_learn(subproblem, use_skel=False)
    #adj = ges_local_learn(subproblem, use_skel=False)
    print(subproblem[1].columns)
    adj = notears_mlp_local_learn(subproblem, use_skel=False)
    print(np.sum(adj!=0))
    results.append(adj)
# with ProcessPoolExecutor(max_workers=nthreads) as executor:
#     for result in executor.map(func_partial, subproblems, chunksize=chunksize):
#         results.append(result)

2025-04-24 12:59:27,445 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/notears/torch/nonlinear.py[line:135] - INFO: GPU is available.
2025-04-24 12:59:27,457 - /homes/shahashka/.conda/envs/lucid_cd/lib/python3.10/site-packages/castle/algorithms/gradient/notears/torch/nonlinear.py[line:237] - INFO: [start]: n=13, d=85, iter_=100, h_=1e-08, rho_=1e+16


Index(['TAPBPL', 'PDIA3', 'DKK3', 'ETS1', 'FOXP1', 'FLI1', 'SHC1', 'CBL',
       'VAV2', 'CTSD', 'ACE', 'CXCR4', 'JUND', 'CD9', 'VAV3', 'E2F1', 'B2M',
       'UBC', 'NFIC', 'ATF2', 'CDR1', 'ETS2', 'RANGAP1', 'ELF1', 'LIN54',
       'BCOR', 'PLCG2', 'MICA', 'GADD45A', 'VCP', 'FYN', 'PIK3CB', 'HMBS',
       'SEC24D', 'CD82', 'WDR5', 'SLC4A2', 'BRD4', 'STAT2', 'MED12', 'KLF4',
       'LNPEP', 'HFE', 'LMNB1', 'ATP8A2', 'ADRA1D', 'ATP8A1', 'CANX', 'DLGAP5',
       'KPNA2', 'UBB', 'IRF1', 'ERG', 'MAP1S', 'HDAC1', 'BHLHE40', 'SLC4A11',
       'STAT5A', 'CALR', 'RAD21', 'ARG2', 'FCGRT', 'GYPC', 'MAF', 'MYC',
       'NOTCH1', 'SLC4A8', 'MAZ', 'CDKN1A', 'IRF2', 'PTPRJ', 'PIK3R1', 'TPST1',
       'PCNA', 'TFRC', 'HLA-A', 'NUSAP1', 'HNRNPC', 'SEC23A', 'AR', 'ICAM1',
       'EGR1', 'ZNF263', 'PROCR', 'radiation'],
      dtype='object')


KeyboardInterrupt: 

In [50]:
from cd_v_partition.fusion import screen_projections, fusion
num_edges = 0
for r in results:
    print(r.shape)
    num_edges += np.sum(r != 0)
print(num_edges)
global_net_non_param = screen_projections(superstructure, custom_partition, results, ss_subset=False, finite_lim=False, data=df_learn.to_numpy())
print(len(global_net_non_param.edges()))
print(len(global_net_non_param.nodes()))

(726, 726)
(93, 93)
(892, 892)
(82, 82)
(209, 209)
(110, 110)
2767
1742
1488


In [ ]:
import networkx as nx
from netgraph import Graph
import matplotlib.pyplot as plt
ground_truth = nx.read_gexf(f"./data/huvec/cd_subgraph_d{dose}.gexf" )
print(ground_truth)
true_positive = 0
for edge in ground_truth.edges:
    if  global_net_non_param.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)

DiGraph with 1487 nodes and 3384 edges
('EGR2', 'POLR2L')
('EGR2', 'TEAD1')
('EGR2', 'POLR2B')
('EGR2', 'POLR2D')
('EGR2', 'WWTR1')
('EGR2', 'SCD5')
('EGR2', 'TTC3')
('EGR2', 'MICU1')
('EGR2', 'TMEM87B')
('EGR2', 'NUP188')
('EGR2', 'RBM39')
('EGR2', 'DONSON')
('EGR2', 'SMPD1')
('EGR2', 'FAF1')
('EGR2', 'ATP6V1G1')
('EGR2', 'LTBP3')
('EGR2', 'NPLOC4')
('EGR2', 'CCNI')
('EGR2', 'TMX4')
('EGR2', 'SLFN11')
('EGR2', 'SH3TC1')
('EGR2', 'BZW2')
('EGR2', 'PNRC1')
('EGR2', 'CAPN1')
('VAV3', 'B2M')
25


In [52]:
radbio_df = pd.read_csv("./data/prior_knowledge/radiation_edges.txt")
radbio_net = nx.DiGraph({edge for edge in zip(radbio_df['node1'], radbio_df['node2'])})
true_positive=0
for edge in radbio_net.edges:
    if  global_net_non_param.has_edge(edge[0], edge[1]):
        true_positive += 1
        print(edge)
print(true_positive)

0


In [53]:
nx.write_gexf(global_net_non_param, f"./data/huvec/cd_dag_gnn_d{dose}.gexf")

In [54]:
len(list(global_net_non_param.neighbors("radiation")))

135

In [55]:
sorted_nodes = sorted(global_net_non_param.out_degree(), key=lambda x: x[1], reverse=True)

# Print nodes and their degree
print("Nodes sorted by degree:")
for node, degree in sorted_nodes:
    print(f"{node}: degree {degree}")

Nodes sorted by degree:
FOXS1: degree 176
radiation: degree 135
EGR2: degree 129
CRYAB: degree 88
ADRA1D: degree 45
BCL2A1: degree 35
COL1A1: degree 33
CD38: degree 31
KLF4: degree 30
NUPR1: degree 28
MAFB: degree 26
IGF2: degree 25
ATP8A2: degree 23
CEP19: degree 21
SMAD6: degree 19
HFE: degree 16
MAF: degree 15
ICAM1: degree 15
FOSB: degree 14
GYPC: degree 13
WDR27: degree 13
KCNMA1: degree 12
VAV3: degree 12
TPST1: degree 12
PTPN7: degree 12
PPP1R14A: degree 10
GLI2: degree 10
BDKRB2: degree 10
SMAD7: degree 9
ACE: degree 9
TMEM184A: degree 9
BCL2L11: degree 9
NFATC2: degree 9
PABPC1L: degree 8
MAP3K14: degree 8
LRRC27: degree 8
TXNIP: degree 7
TRPC6: degree 7
CCNA1: degree 7
ATP8A1: degree 7
PTPN22: degree 6
EGR1: degree 6
PTGS2: degree 6
PBX3: degree 6
SLC4A11: degree 6
PTPRJ: degree 6
CCND2: degree 6
CDKN2C: degree 6
SETBP1: degree 5
VCAN: degree 5
ATF3: degree 5
LIN54: degree 5
SLC4A8: degree 5
ZNF263: degree 5
SIX2: degree 5
STAG3: degree 5
GADD45B: degree 5
LMO2: degree 5
GATA